1. Design Model (input, output size, forward pass)
2. Construct loss and optimizer
3. Training loop
    - forward pass : compute prediction
    - backward pass : gardients
    - update weights
    - zero the gradient values

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


0. prepare data

In [2]:
bc = datasets.load_breast_cancer()
bc

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]], shape=(569, 30)),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,

In [3]:
X, y = bc.data, bc.target

In [4]:
n_samples, n_features = X.shape

print(n_samples, n_features)

569 30


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

Scale

In [6]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X_train.dtype

dtype('float64')

In [8]:
# converting to tensors

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# convert the output values(y) the row form to column form
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)



1. define model

In [9]:
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


In [28]:
# f = mx + b, sigmoid at end

class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted


model = LogisticRegression(n_features)

2. loss and optimizer

In [29]:
learning_rate = 0.01
criterian = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

3. training loop

In [30]:
epochs = 100

for epoch in range(epochs):
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterian(y_predicted, y_train)

    #backward
    loss.backward()

    #update
    optimizer.step()

    #zero_grad()
    optimizer.zero_grad()

    #printing the epochs and loss
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch}, loss: {loss.item():.4f}')


epoch: 9, loss: 0.8329
epoch: 19, loss: 0.6243
epoch: 29, loss: 0.5045
epoch: 39, loss: 0.4310
epoch: 49, loss: 0.3816
epoch: 59, loss: 0.3459
epoch: 69, loss: 0.3187
epoch: 79, loss: 0.2970
epoch: 89, loss: 0.2794
epoch: 99, loss: 0.2646


Validation

In [31]:
#with no grad that should not claculate the gradients while validating

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round() #that roud probabilities, gives the 0 or 1 sigmoid generates tehe values  is b/w 0 abd 1 
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')


accuracy = 0.8860
